# Trabalho 1 - MATA62 - Engenharia de Software I
Componentes : Gabriel Andrade, Ana Clara Batista, Caroline Alves, Jeisiane Macedo

In [1]:
import pandas as pd
from settings import *
from scrape import *
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from ipywidgets import interact
import swifter
import ipywidgets as widgets
from datetime import datetime
%matplotlib inline

In [ ]:
scrapePage(anacUrl)

In [2]:
situacaoVoos = []
fileCsvFinal = pd.DataFrame()
for ano in range(2015, 2020):
    print(f"Lendo os arquivos do ano {ano}")
    for i in range(1, 13):
            sep = [';', ',','\t']
            sepIndex = 0
            while True:
                try:
                    fileCsv = pd.read_csv(
                        f'data/{ano}-{i:02}.csv', encoding='ISO-8859-1', sep=sep[sepIndex], low_memory=False)
                    if ('internet' in fileCsv.columns[0]):
                        fileCsv.columns = fileCsv.iloc[0]
                        fileCsv = fileCsv[1:]
                    if len(fileCsv.columns) <= 1:
                        raise ValueError('sepIndex')
                    else:
                        try:
                            fileCsv = fileCsv.drop(columns='Data Prevista')
                        except:
                            pass
                        fileCsv.columns = ['ICAO Empresa Aérea', 'Número Voo', 'Código DI', 'Código Tipo Linha',
                                        'ICAO Aeródromo Origem', 'ICAO Aeródromo Destino', 'Partida Prevista',
                                        'Partida Real', 'Chegada Prevista', 'Chegada Real', 'Situação Voo',
                                        'Código Justificativa']
                    
                    fileCsv = fileCsv.assign(Ano = ano)
                    fileCsv = fileCsv.assign(Mes = i)
                    
                    fileCsvFinal = pd.concat([fileCsvFinal, fileCsv])
                    print(len(fileCsvFinal))
                    break
                except Exception as e:
                    if e.args[0] == 'sepIndex':
                        if (sepIndex < 3):
                            sepIndex += 1
                    else:
                        print(e)
fileCsvFinal['Situação Voo'] = fileCsvFinal['Situação Voo'].str.upper()
fileCsvFinal['Situação Voo'] = fileCsvFinal['Situação Voo'].str.replace('NÃO REALIZADO', 'CANCELADO', regex=False)

Lendo os arquivos do ano 2015
114636
216629
327636
434556
542910
648608
760762
868806
976224
1085370
1189754
1301570
Lendo os arquivos do ano 2016
1415829
1517497
1619362
1718322
1810178
1898218
1990804
2082306
2168746
2258370
2345436
2440576
Lendo os arquivos do ano 2017
2539374
2624374
2716233
2802437
2893114
2980986
3077896
3172081
3261534
3355080
3446623
3546060
Lendo os arquivos do ano 2018
3649557
3739167
3829956
3909737
3993491
4073793
4164009
4248397
4329316
4412963
4492988
4580616
Lendo os arquivos do ano 2019
4672433
4751959
4835021
4912465
4989166
5063355
5149027
5229616
5310153
5394392
5475390
5563607


In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
fig.suptitle("Dinâmica anual de vôos realizados/cancelados", fontsize=20)
SituacaoPorAno = pd.DataFrame()
for situacao in fileCsvFinal['Situação Voo'].unique():
    df = fileCsvFinal.loc[fileCsvFinal['Situação Voo'] == situacao]
    df = df['Ano'].value_counts()
    SituacaoPorAno= pd.concat([SituacaoPorAno, df], axis=1)
SituacaoPorAno.columns = ['Realizado', 'Cancelado']
graphRealizados = plt.bar(SituacaoPorAno['Realizado'].index, SituacaoPorAno['Realizado'].values, color='#740000', label = 'Realizados')
graphCancelados = plt.bar(SituacaoPorAno['Cancelado'].index, SituacaoPorAno['Cancelado'].values, color='#5b616d', label = 'Cancelados')

def autolabel(graph, color):
  
    for rect in graph:
        valor = rect.get_height()
        ax.annotate(f'{valor}',
                    xy=(rect.get_x() + rect.get_width() / 2, valor),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    color=color,
                    fontsize=12)

plt.ylabel('Quantidade de vôos (Milhão)')
plt.xlabel('Ano')

plt.legend(handles=[graphRealizados, graphCancelados])

autolabel(graphRealizados, '#000')
autolabel(graphCancelados, '#FFF')

In [ ]:
colorPastel = plt.cm.Pastel1(np.arange(12))
# colorBlack

def autolabel(graph, color):
  
    for rect in graph:
        valor = rect.get_height()
        ax.annotate(f'{valor}',
                    xy=(rect.get_x() + rect.get_width() / 2, valor),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    color=color,
                    fontsize=12)
def f(ano):
    fig, ax = plt.subplots(figsize=(14, 7))
    fig.suptitle("Dinâmica mensal de vôos Realizados/Cancelados", fontsize=20)
    if(ano < 2015):
        df = fileCsvFinal
    else:
        df = fileCsvFinal.loc[fileCsvFinal['Ano'] == ano]
    SituacaoPorMes = pd.DataFrame()
    for mes in range(1, 13):
        df2 = df.loc[df['Mes'] == mes]
        df2 = df2['Situação Voo'].value_counts()
        SituacaoPorMes= pd.concat([SituacaoPorMes, df2], axis=1)
    SituacaoPorMes.columns = ['Janeiro','Fevereiro','Março','Abril','Maio', 'Junho','Julho', 'Agosto','Setembro','Outubro', 'Novembro', 'Dezembro']
    
    x = plt.bar(SituacaoPorMes.iloc[0].index, SituacaoPorMes.iloc[0].values,  color='#740000', label = 'Realizados')
    y = plt.bar(SituacaoPorMes.iloc[1].index, SituacaoPorMes.iloc[1].values,  color='#5b616d', label = 'Cancelados')
        
    plt.ylabel('Quantidade de vôos')
    plt.xlabel('Mês')
    plt.legend(handles=[graphRealizados, graphCancelados])

    labels = ax.get_xticklabels()
    for label in labels:
        label.set_rotation(45)
    print(f"Média de voos realizados : {np.around(int(SituacaoPorMes.iloc[0].mean()))}")
    print(f"Média de voos cancelados : {np.around(int(SituacaoPorMes.iloc[1].mean()))}")
    autolabel(x, '#000')
    autolabel(y, "#FFF")



interact(f, ano=widgets.IntSlider(min=2014,max=2019,step=1,value=2014))

## Participação das principais empresas no cenário aéreo brasileiro
ASDHLASDJLD <BR>
AÇSDKASDLKLASD 
    <br>
ASLDKLAKSDKLAÇSD
    <br>
ALSDÇAÇSDKÇ
ASDLÇKASDKASÇLD

In [ ]:
explode = [0.05]
pieColors=['#ff0000', '#5b616d', '#7a3f7a', '#80a07a', '#c3867a', '#55e5e5']
def f(ano, mes, empresas):
    explode = [0.05]
    for i in range(empresas):
        explode.append(0)
    fig = plt.figure(figsize=(8,5))

    title = f"Participação das {empresas} empresas com maior número de voos"
    if (ano >= 2015) :
        voosPorAno = fileCsvFinal.loc[fileCsvFinal['Ano'] == ano]
        title += f" em {ano}"
    else:
        voosPorAno = fileCsvFinal
    voosRealizados = voosPorAno[voosPorAno['Situação Voo'] == 'REALIZADO']
    voosCancelados = voosPorAno[voosPorAno['Situação Voo'] == 'CANCELADO']
    if (mes >= 1 and mes <= 12):
        title += f" no mês {mes}"
        voosRealizados = voosRealizados.loc[voosRealizados['Mes'] == mes]['ICAO Empresa Aérea'].value_counts()
        voosCancelados = voosCancelados.loc[voosCancelados['Mes'] == mes]['ICAO Empresa Aérea'].value_counts()
    else:
        voosRealizados = voosRealizados['ICAO Empresa Aérea'].value_counts()
        voosCancelados = voosCancelados['ICAO Empresa Aérea'].value_counts()
    fig.suptitle('Participação das principais empresas no cenário aéreo brasileiro', fontsize=20)


    outrasEmpresas = voosRealizados[voosRealizados < voosRealizados[empresas - 1]]
    outrasEmpresasRealizadosDf = pd.Series({'Outros' : outrasEmpresas.sum()})
    voosRealizadosDF = pd.concat([voosRealizados[:empresas], outrasEmpresasRealizadosDf])

    outrasEmpresasCancelados = voosCancelados[voosCancelados < voosCancelados[empresas - 1]]
    outrasEmpresasCanceladosDf = pd.Series({'Outros' : outrasEmpresasCancelados.sum()})
    voosCanceladosDF = pd.concat([voosCancelados[:empresas], outrasEmpresasCanceladosDf])
    
    
    ax1 = plt.subplot2grid((1,2),(0,0))
    ax1.set_title("Realizados")
    plt.pie(voosRealizadosDF, autopct='%1.1f%%', explode=explode, shadow=True, labels=voosRealizadosDF.index, colors=pieColors)
    
    ax2 = plt.subplot2grid((1,2),(0,1))
    ax2.set_title("Cancelados")
    plt.pie(voosCanceladosDF, autopct='%1.1f%%', explode=explode, shadow=True, labels=voosCanceladosDF.index, colors=pieColors)
interact(f, ano=widgets.IntSlider(min=2014,max=2019,step=1,value=2014), mes=widgets.IntSlider(min=0,max=12,step=1,value=0), empresas=widgets.IntSlider(min=2,max=5,step=1,value=2))

In [ ]:
def f(ano, mes):
    fig, ax = plt.subplots(figsize=(10,6))
    fig.suptitle("Tipos de vôos", fontsize=20)
    if (ano >= 2015) :
        linhas = fileCsvFinal.loc[fileCsvFinal['Ano'] == ano]
    else:
        linhas = fileCsvFinal
    if (mes >= 1 and mes <= 12):
        linhas = linhas.loc[linhas['Mes'] == mes]['Código Tipo Linha'].value_counts()
    else:
        linhas = linhas['Código Tipo Linha'].value_counts()
    tiposDeVoos = plt.bar(linhas.index[:], linhas.values[:], color='#7a3f7a')
    def autolabel(graph, color):
    
        for rect in graph:
            valor = rect.get_height()
            ax.annotate(f'{valor}',
                      xy=(rect.get_x() + rect.get_width() / 2, valor),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center', va='bottom',
                      color=color,
                      fontsize=12)
    autolabel(tiposDeVoos, '#000')
  # labels = ax.get_xticklabels()
  # for label in labels:
  #     label.set_rotation(45)
    print("N - Nacional\nR - Regional\nI = Internacional\nE - Especial\nG - Cargueiro Internacional\nC - Cargueiro\nL - Rede Postal\nH - Sub-Regional")
interact(f, ano=widgets.IntSlider(min=2014,max=2019,step=1,value=2015),mes=widgets.IntSlider(min=0,max=12,step=1,value=0))

In [ ]:
def f(aeroportos, ano, mes):
    fig, axs = plt.subplots(nrows=1, figsize=(10, 5))
    fig.suptitle(f'Concentração dos {aeroportos} aeroportos com maior número de partida de vôos - 2015 a 2019', fontsize=20)
    voosRealizados = fileCsvFinal.loc[fileCsvFinal['Situação Voo'] == 'REALIZADO']
    
    if (ano >= 2015) :
        aeroportosComMaisVoos = voosRealizados.loc[voosRealizados['Ano'] == ano]
    else:
        aeroportosComMaisVoos = voosRealizados
    if (mes >= 1 and mes <= 12):
        aeroportosComMaisVoos = aeroportosComMaisVoos.loc[aeroportosComMaisVoos['Mes'] == mes]['ICAO Aeródromo Origem'].value_counts()
    else:
        aeroportosComMaisVoos = aeroportosComMaisVoos['ICAO Aeródromo Origem'].value_counts()
    
    aeroportosComMaisVoos[:aeroportos].plot(kind='bar')
    labels = ax.get_xticklabels()
    for label in labels:
        label.set_rotation(45)
interact(f, aeroportos=widgets.IntSlider(min=5,max=30,step=1,value=5), ano=widgets.IntSlider(min=2014,max=2019,step=1,value=2014),mes=widgets.IntSlider(min=0,max=12,step=1,value=0))

In [ ]:
def f(aeroportos, ano, mes):
    fig, axs = plt.subplots(nrows=1, figsize=(10, 5))
    fig.suptitle(f'Concentração dos {aeroportos} aeroportos com maior número de chegada de vôos', fontsize=20)
    voosRealizados = fileCsvFinal.loc[fileCsvFinal['Situação Voo'] == 'REALIZADO']
    
    if (ano >= 2015) :
        aeroportosComMaisVoos = voosRealizados.loc[voosRealizados['Ano'] == ano]
    else:
        aeroportosComMaisVoos = voosRealizados
    if (mes >= 1 and mes <= 12):
        aeroportosComMaisVoos = aeroportosComMaisVoos.loc[aeroportosComMaisVoos['Mes'] == mes]['ICAO Aeródromo Destino'].value_counts()
    else:
        aeroportosComMaisVoos = aeroportosComMaisVoos['ICAO Aeródromo Destino'].value_counts()
    
    aeroportosComMaisVoos[:aeroportos].plot(kind='bar')
    labels = ax.get_xticklabels()
    for label in labels:
        label.set_rotation(45)
interact(f, aeroportos=widgets.IntSlider(min=5,max=30,step=1,value=5), ano=widgets.IntSlider(min=2014,max=2019,step=1,value=2015),mes=widgets.IntSlider(min=0,max=12,step=1,value=0))

In [3]:
#Tratamento de dados de Horário de partida
voosRealizados = fileCsvFinal.loc[(fileCsvFinal['Situação Voo'] == 'REALIZADO')]
 
#Vericar a quantidade de dados nulos
#voosRealizados.isnull().sum()
 
#Verificar o tamanho das Strings dos dados a fim de verificar divergências nas informações
#serie = voosRealizados['Partida Prevista'].str.len()
#serie.groupby(serie, dropna = False).size()
 
#Filtra o df apenas para as linhas que possuem valores em Partida Prevista e Partida Real
voosRealizados = voosRealizados[(voosRealizados['Partida Prevista'].str.len()>=14) & (voosRealizados['Partida Real'].str.len()>=14)]
 
#Converte data em string para data em datetime
partida_prevista = voosRealizados.apply(lambda x: datetime.strptime(x['Partida Prevista'], '%d/%m/%Y %H:%M'), axis = 1)
partida_real = voosRealizados.apply(lambda x: datetime.strptime(x['Partida Real'], '%d/%m/%Y %H:%M'), axis = 1)
 
voosRealizados['Partida Prevista'] = partida_prevista
voosRealizados['Partida Real'] = partida_real

KeyboardInterrupt: 

In [24]:
voosRealizados = fileCsvFinal.loc[(fileCsvFinal['Situação Voo'] == 'REALIZADO')]
voosRealizados = voosRealizados[(voosRealizados['Partida Prevista'].str.len()>=14) & (voosRealizados['Partida Real'].str.len()>=14)]

In [25]:
# voosRealizados['Partida Prevista'].astype('datetime64') - voosRealizados['Partida Real'].astype('datetime64')
def x(previsto):
    return(datetime.strptime(previsto, '%d/%m/%Y %H:%M'))
voosRealizados['Partida Prevista'] = voosRealizados['Partida Prevista'].swifter.apply(x)
voosRealizados['Partida Real'] = voosRealizados['Partida Real'].swifter.apply(x)

KeyboardInterrupt: 

In [20]:
# dif = voosRealizados.apply(lambda x: (x['Partida Real'] - x['Partida Prevista']), axis = 1)
def subtract(df):
    return(df['Partida Real'] - df['Partida Prevista'])
dif2 = voosRealizados.swifter.apply(subtract, axis=1)

In [22]:
dif2

0         0 days 00:17:00
1       -1 days +23:46:00
2       -1 days +23:50:00
3         0 days 00:18:00
4       -1 days +23:46:00
               ...       
88212     0 days 00:09:00
88213   -1 days +23:57:00
88214     0 days 00:37:00
88215     0 days 00:03:00
88216   -1 days +23:51:00
Length: 4893323, dtype: timedelta64[ns]

In [ ]:
#Cálculo de tempo de atraso
#cria uma lista com a diferença dos valores de partida
dif = voosRealizados.apply(lambda x: (x['Partida Real'] - x['Partida Prevista']), axis = 1)
print('.')

#converte os horários do dataFrame dif para minutos inteiros e 
#zera os dados caso o vôo saiu adiantado
def datetime_to_int(x):
    if x.days < 0:
        return 0
    else:
        return int(x.seconds//60)

dif_minutos = dif.apply(datetime_to_int)
print('..')

#Adiciona a série com os valores de atraso como uma coluna
dadosdeAtraso = voosRealizados.assign(Atraso = dif_minutos)
print('...')

#Salva em um arquivo csv local
dadosdeAtraso.to_csv("data/dadosdeAtrasoTratados.csv", ';', index = False)

In [ ]:
# dadosdeAtraso = pd.read_csv("data/dadosdeAtrasoTratados.csv", ';', low_memory = False)

In [ ]:
explode = (0.1, 0, 0, 0, 0)
pieColors=['#ff0000', '#5b616d', '#7a3f7a', '#80a07a', '#c3867a']

def f(ano, minutos):
    dadosdeAtrasoAno = dadosdeAtraso[dadosdeAtraso['Ano'] == ano]
    dadosdeAtrasoX = dadosdeAtrasoAno[dadosdeAtrasoAno['Atraso'] >= minutos]
    if minutos < 120:
        title = f"Atrasos entre {minutos} e {minutos + 20} minutos"
        dadosdeAtrasoX = dadosdeAtrasoX[dadosdeAtrasoX['Atraso'] < (minutos + 20)]['ICAO Empresa Aérea'].value_counts()
    else:
        title = f"Atrasos com mais de {minutos} minutos"
        dadosdeAtrasoX = dadosdeAtrasoX['ICAO Empresa Aérea'].value_counts()
    outrasEmpresas = dadosdeAtrasoX[dadosdeAtrasoX < dadosdeAtrasoX[3]]
    outrasEmpresasAtrasoX = pd.Series({'Outros' : outrasEmpresas.sum()})
    dadosdeAtrasoXDF = pd.concat([dadosdeAtrasoX[:4], outrasEmpresasAtrasoX])

    fig1, ax1 = plt.subplots(figsize=(10,5))
    fig1.suptitle(f"Empresas Aéreas com maior índice de atrasos", fontsize=20)
    ax1.set_title(title)
    ax1.pie(dadosdeAtrasoXDF, autopct='%1.1f%%', shadow=True, labels = dadosdeAtrasoXDF.index, startangle=90, explode = explode, colors=pieColors)
    ax1.axis('equal')

    plt.show()

interact(f, ano=widgets.IntSlider(min=2015,max=2019,step=1,value=2015), minutos=widgets.IntSlider(min=20,max=120,step=20,value=20))